# Extract Meta-data - pdfplumber 
* Filename
* DMS No. 
* Version
* Date (Doc Hist)
* Date (Exp from DMS)
* Doc type 
* Classification

In [12]:
import pandas as pd
import os
import pdfplumber
import re

# Path to local PDF files
pdf_folder = r"C:\Users\emlu\OneDrive - Netcompany\Desktop\Virtual Tech Vestas\RAG_LLM_POC_v1\data\Vestas_RTP\Documents\Documents"

# Create Regex Patterns
dms_pattern = re.compile(r"\b\d{4}-\d{4}\b")
ver_pattern = re.compile(r"\bVER[:\s]+(\d{2})\b", re.IGNORECASE)
date_pattern = re.compile(r"Date[:\s]*([\d]{4}-[\d]{2}-[\d]{2})", re.IGNORECASE)
exported_date_pattern = re.compile(r"Exported from DMS.*?(\d{4}-\d{2}-\d{2})", re.IGNORECASE)
doc_type_pattern = re.compile(r"\bT\d{2}\b")
classification_pattern = re.compile(r"\b(Confidential|Restricted)\b(?![a-zA-Z])", re.IGNORECASE)

# Results list
results = []

for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)

        with pdfplumber.open(filepath) as pdf:
            text = pdf.pages[0].extract_text() or ""

        # Metadata extraction
        dms_match = dms_pattern.search(text)
        ver_match = ver_pattern.search(text)
        exported_match = exported_date_pattern.search(text)
        doc_type_match = doc_type_pattern.search(text)
        classification_match = classification_pattern.search(text)
        date_match = date_pattern.search(text)

        result = {
            "filename": filename,
            "DMS No.": dms_match.group(0) if dms_match else None,
            "Ver": ver_match.group(1) if ver_match else None,
            "Date (Doc hist)": date_match.group(1) if date_match else None,
            "Date (Exported from DMS)": exported_match.group(1) if exported_match else None,
            "Doc type": doc_type_match.group(0) if doc_type_match else None,
            "Classification": classification_match.group(1).capitalize() if classification_match else None
        }

        results.append(result)

# Convert to DataFrame
df_pdfplumber = pd.DataFrame(results)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [13]:
#Checking Output
df_pdfplumber

,filename,DMS No.,Ver,Date (Doc hist),Date (Exported from DMS),Doc type,Classification
0,0078-6200_V07 - 0078-6200_4MW Mk3E Setting and...,0078-6200,07,None,None,None,Confidential
1,0079-6646 09. SII-Nacelle (yearly).pdf,0079-6646,None,2024-07-25,None,T09,Confidential
2,0109-7505_V05 - TIS_4MW Mk3 Ready to Protect C...,0109-7505,None,None,None,T12,Confidential
3,0138-1002_V04 - Battery Cell Impedance Testing...,0138-1002,None,2023-09-05,None,T09,Confidential
4,Firmware revision 5.2 in the 230 V UPS.pdf,0088-1273,None,2019-12-12,None,T09,Restricted
5,Installation and operation - Smart-UP RT.PDF,None,None,None,None,None,Confidential
6,Rotor locking system.pdf,0077-2293,None,2022-12-03,None,T09,Restricted
7,"SM,CIM4449, repl. battery and cable.pdf",0109-6919,None,2021-11-16,None,None,Confidential
8,TIS_4MW RtoP Recovery Flow Chart (Mk3A-B-E).pdf,None,None,None,None,None,Confidential
9,User guide for the ready-to_x0002_protect (Rto...,0079-9386,None,2019-02-28,None,T09,Restricted


### Doc type using pdf plumber

In [17]:
import pandas as pd
import os
import pdfplumber
import re

# Path to local PDF files
pdf_folder = r"C:\Users\emlu\OneDrive - Netcompany\Desktop\Virtual Tech Vestas\RAG_LLM_POC_v1\data\Vestas_RTP\Documents\Documents"

# Create Regex Patterns
doc_type_pattern = re.compile(r"\bT\d{2}\b")

# Results list
results = []

for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)

        with pdfplumber.open(filepath) as pdf:
            text = pdf.pages[0].extract_text() or ""

        # Metadata extraction
        doc_type_match = doc_type_pattern.search(text)

        result = {
            "filename": filename,
            "Doc type": doc_type_match.group(0) if doc_type_match else None
        }

        results.append(result)

# Convert to DataFrame
df_doc_type_pdfplumber = pd.DataFrame(results)

# Check Output 
df_doc_type_pdfplumber

CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

,filename,Doc type
0,0078-6200_V07 - 0078-6200_4MW Mk3E Setting and...,None
1,0079-6646 09. SII-Nacelle (yearly).pdf,T09
2,0109-7505_V05 - TIS_4MW Mk3 Ready to Protect C...,T12
3,0138-1002_V04 - Battery Cell Impedance Testing...,T09
4,Firmware revision 5.2 in the 230 V UPS.pdf,T09
5,Installation and operation - Smart-UP RT.PDF,None
6,Rotor locking system.pdf,T09
7,"SM,CIM4449, repl. battery and cable.pdf",None
8,TIS_4MW RtoP Recovery Flow Chart (Mk3A-B-E).pdf,None
9,User guide for the ready-to_x0002_protect (Rto...,T09


# Extract Meta-data - Tesseract (OCR)
* Filename
* DMS No. 
* Version
* Date (Doc Hist)
* Date (Exp from DMS)
* Doc type 
* Classification

In [9]:
import pandas as pd 
import os
import fitz  # PyMuPDF
import pytesseract  # OCR
from PIL import Image
import io
import re

# Path to local PDF files
pdf_folder = r"C:\Users\emlu\OneDrive - Netcompany\Desktop\Virtual Tech Vestas\RAG_LLM_POC_v1\data\Vestas_RTP\Documents\Documents"

# Create Regex Patterns
dms_pattern = re.compile(r"\b\d{4}-\d{4}\b")
ver_pattern = re.compile(r"\bVER[:\s]+(\d{2})\b", re.IGNORECASE)
date_pattern = re.compile(r"Date[:\s]*([\d]{4}-[\d]{2}-[\d]{2})", re.IGNORECASE)
exported_pattern = re.compile(r"Exported from DMS.*?(\d{4}-\d{2}-\d{2})", re.IGNORECASE)
doc_type_pattern = re.compile(r"\bT\d{2}\b")
classification_pattern = re.compile(r"\b(Confidential|Restricted)\b(?![a-zA-Z])", re.IGNORECASE)
approved_pattern = re.compile(r"\bApproved\b", re.IGNORECASE)

# Results list
results = []

for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)
        doc = fitz.open(filepath)

        # Take the first page as an image
        pix = doc[0].get_pixmap(dpi=500)
        img_data = Image.open(io.BytesIO(pix.tobytes("png")))

        # OCR with pytesseract
        ocr_text = pytesseract.image_to_string(img_data)

        # Metadata extraction
        dms_match = dms_pattern.search(ocr_text)
        ver_match = ver_pattern.search(ocr_text)
        date_match = date_pattern.search(ocr_text)
        exported_match = exported_pattern.search(ocr_text)
        doc_type_match = doc_type_pattern.search(ocr_text)
        classification_match = classification_pattern.search(ocr_text)
        approved_match = approved_pattern.search(ocr_text)

        result = {
            "filename": filename,
            "DMS No.": dms_match.group(0) if dms_match else None,
            "Ver": ver_match.group(1) if ver_match else None,
            "Date (Doc hist)": date_match.group(1) if date_match else None,
            "Date (Exported from DMS)": exported_match.group(1) if exported_match else None,
            "Doc type": doc_type_match.group(0) if doc_type_match else None,
            "Classification": classification_match.group(1).capitalize() if classification_match else None,
            "Approved": "Y" if approved_match else "N" 
        }

        results.append(result)

# Convert to DataFrame
df_ocr = pd.DataFrame(results)


In [11]:
# Check output
df_ocr

,filename,DMS No.,Ver,Date (Doc hist),Date (Exported from DMS),Doc type,Classification,Approved
0,0078-6200_V07 - 0078-6200_4MW Mk3E Setting and...,0078-6200,07,None,2024-12-02,T07,Confidential,Y
1,0079-6646 09. SII-Nacelle (yearly).pdf,0079-6646,10,2024-07-25,2024-10-24,None,Confidential,Y
2,0109-7505_V05 - TIS_4MW Mk3 Ready to Protect C...,0109-7505,05,None,2024-10-15,T09,Confidential,Y
3,0138-1002_V04 - Battery Cell Impedance Testing...,0138-1002,04,2023-09-05,2024-12-02,T09,Confidential,Y
4,Firmware revision 5.2 in the 230 V UPS.pdf,0088-1273,00,2019-12-12,2024-12-11,None,Restricted,Y
5,Installation and operation - Smart-UP RT.PDF,0030-8011,00,None,2024-12-11,None,Restricted,Y
6,Rotor locking system.pdf,0077-2293,03,2022-12-03,2025-03-03,T09,Restricted,Y
7,"SM,CIM4449, repl. battery and cable.pdf",0109-6919,01,2021-11-16,2024-12-12,None,Confidential,Y
8,TIS_4MW RtoP Recovery Flow Chart (Mk3A-B-E).pdf,0109-7505,05,None,2024-10-15,None,Restricted,Y
9,User guide for the ready-to_x0002_protect (Rto...,0079-9386,00,2019-02-28,2024-10-15,None,Restricted,Y


# Extract Meta-data - Hybrid approach 
* Filename
* DMS No. 
* Version
* Date (Doc Hist)
* Date (Exp from DMS)
* Doc type 
* Classification
* Approved (Y/N)

In [ ]:
import pandas as pd
import os
import fitz  # PyMuPDF
import pytesseract  # OCR
from PIL import Image
import io
import re
import pdfplumber

# Path to local PDF files
pdf_folder = r"C:\Users\emlu\OneDrive - Netcompany\Desktop\Virtual Tech Vestas\RAG_LLM_POC_v1\data\Vestas_RTP\Documents\Documents"

# Create Regex Patterns
dms_pattern = re.compile(r"\b\d{4}-\d{4}\b")
ver_pattern = re.compile(r"\bVER[:\s]+(\d{2})\b", re.IGNORECASE)
date_pattern = re.compile(r"Date[:\s]*([\d]{4}-[\d]{2}-[\d]{2})", re.IGNORECASE)
exported_pattern = re.compile(r"Exported from DMS.*?(\d{4}-\d{2}-\d{2})", re.IGNORECASE)
doc_type_pattern = re.compile(r"\bT\d{2}\b")
classification_pattern = re.compile(r"\b(Confidential|Restricted)\b(?![a-zA-Z])", re.IGNORECASE)
approved_pattern = re.compile(r"\bApproved\b", re.IGNORECASE)

# Results list
results = []

for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)

        # Step 1: Try extracting "Doc type" using pdfplumber
        doc_type_value = None
        try:
            with pdfplumber.open(filepath) as pdf:
                plumber_text = pdf.pages[0].extract_text() or ""
                doc_type_match_pdf = doc_type_pattern.search(plumber_text)
                if doc_type_match_pdf:
                    doc_type_value = doc_type_match_pdf.group(0)
        except Exception as e:
            print(f"pdfplumber failed on {filename}: {e}")

        # Step 2: Use OCR for everything (including Doc type if not already found)
        doc = fitz.open(filepath)
        pix = doc[0].get_pixmap(dpi=500)
        img_data = Image.open(io.BytesIO(pix.tobytes("png")))
        ocr_text = pytesseract.image_to_string(img_data)

        # Extract metadata
        dms_match = dms_pattern.search(ocr_text)
        ver_match = ver_pattern.search(ocr_text)
        date_match = date_pattern.search(ocr_text)
        exported_match = exported_pattern.search(ocr_text)
        classification_match = classification_pattern.search(ocr_text)
        approved_match = approved_pattern.search(ocr_text)

        # Only fallback to OCR for doc_type if not found by pdfplumber
        if not doc_type_value:
            doc_type_match_ocr = doc_type_pattern.search(ocr_text)
            if doc_type_match_ocr:
                doc_type_value = doc_type_match_ocr.group(0)

        result = {
            "filename": filename,
            "DMS No.": dms_match.group(0) if dms_match else None,
            "Ver": ver_match.group(1) if ver_match else None,
            "Date (Doc hist)": date_match.group(1) if date_match else None,
            "Date (Exported from DMS)": exported_match.group(1) if exported_match else None,
            "Doc type": doc_type_value,
            "Classification": classification_match.group(1).capitalize() if classification_match else None,
            "Approved": "Y" if approved_match else "N"
        }

        results.append(result)

# Convert to DataFrame
df_hybrid = pd.DataFrame(results)


In [ ]:
# Check Output
df_hybrid

,filename,DMS No.,Ver,Date (Doc hist),Date (Exported from DMS),Doc type,Classification,Approved
0,0078-6200_V07 - 0078-6200_4MW Mk3E Setting and...,0078-6200,07,None,2024-12-02,T07,Confidential,Y
1,0079-6646 09. SII-Nacelle (yearly).pdf,0079-6646,10,2024-07-25,2024-10-24,T09,Confidential,Y
2,0109-7505_V05 - TIS_4MW Mk3 Ready to Protect C...,0109-7505,05,None,2024-10-15,T12,Confidential,Y
3,0138-1002_V04 - Battery Cell Impedance Testing...,0138-1002,04,2023-09-05,2024-12-02,T09,Confidential,Y
4,Firmware revision 5.2 in the 230 V UPS.pdf,0088-1273,00,2019-12-12,2024-12-11,T09,Restricted,Y
5,Installation and operation - Smart-UP RT.PDF,0030-8011,00,None,2024-12-11,None,Restricted,Y
6,Rotor locking system.pdf,0077-2293,03,2022-12-03,2025-03-03,T09,Restricted,Y
7,"SM,CIM4449, repl. battery and cable.pdf",0109-6919,01,2021-11-16,2024-12-12,None,Confidential,Y
8,TIS_4MW RtoP Recovery Flow Chart (Mk3A-B-E).pdf,0109-7505,05,None,2024-10-15,None,Restricted,Y
9,User guide for the ready-to_x0002_protect (Rto...,0079-9386,00,2019-02-28,2024-10-15,T09,Restricted,Y
